In [5]:
def query(sqlString:str)->None:
    
    try:
        return duckdb.query(sqlString).show(max_width = 100000, max_rows = 100000)
    except Exception as e:
        return e


In [3]:
def getInvoices():
    
    try:
        relativeInvoicesURL = '/v1/invoices'
        response = json.loads(
        rs.get(
            f"{base_url}{relativeInvoicesURL}",
            headers=HTTPheaders)
        .content)

        return response['items']
    
    except Exception as e:
        return e

In [4]:
def getOneTimeInvoiceLineItems(invoiceID:str)->dict:
    
    try:
        newCommerceOneTimeBillingURL = f'/v1/invoices/OneTime-{invoiceID}/lineitems/OneTime/BillingLineItems?size=5000'

        response = json.loads(
            rs.get(
                    f"{base_url}{newCommerceOneTimeBillingURL}",
                    headers=HTTPheaders)
            .content) if rs.get(
                    f"{base_url}{newCommerceOneTimeBillingURL}",
                    headers=HTTPheaders).status_code == 200 else None # if response status code = 200
        
        return response['items']
    
    except Exception as e:
        return e

In [65]:

import requests as rs
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
from azure.storage.blob import ContainerClient,BlobServiceClient
from io import StringIO
import numpy as np
import json
import datetime as dt
import warnings
import duckdb as db
import time

# Set display options

pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

cxn = db.connect('../DB/dbfile')

print( 'connected to Db')
time.sleep(1)

# Parse Secrets

try:
    secrets = json.load(open(file='../secrets.json'))
    print('Secrets file found.')
except:
    print('Secrets file not found or some error in file. Make sure the file exists in the same directory as the code.')

if secrets:
    refresh_token = secrets['refresh_token']

    app_id = secrets['app_id']

    app_secret = secrets['app_secret']

    request_body = {
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
            "scope": "openid",
            "resource": "https://api.partnercenter.microsoft.com",
            "client_id": app_id,
            "client_secret": app_secret,
        }

    base_url = (
    'https://api.partnercenter.microsoft.com'
    )


    isAccessTokenObtained = 0


try:
    req = rs.post(
                "https://login.windows.net/6e75cca6-47f0-47a3-a928-9d5315750bd9/oauth2/token"
                ,data = request_body)
    
    access_token = json.loads(req.text)['access_token']
    isAccessTokenObtained = 1
    print('Refresh Token valid, access token obtained. \nAccess token : ',access_token[:20]+'...')
except:
    print('Refresh token expired, or no internet connection. \nUnable to obtain access token')
    print(json.loads(req.text))
    

if isAccessTokenObtained == 1:
        
        HTTPheaders = {'Authorization': 'Bearer ' + access_token}


connected to Db
Secrets file found.
Refresh token expired, or no internet connection. 
Unable to obtain access token
{'error': 'invalid_grant', 'error_description': "AADSTS65001: The user or administrator has not consented to use the application with ID '006568e8-96fd-4d41-89cb-b90ad6d3ae7f' named 'Partner Center Web App'. Send an interactive authorization request for this user and resource. Trace ID: 3c0f51c2-bdfe-46b5-bf3c-9d319af63000 Correlation ID: 596d010e-1a6f-42a3-b917-ad161edb5975 Timestamp: 2024-02-15 14:11:08Z", 'error_codes': [65001], 'timestamp': '2024-02-15 14:11:08Z', 'trace_id': '3c0f51c2-bdfe-46b5-bf3c-9d319af63000', 'correlation_id': '596d010e-1a6f-42a3-b917-ad161edb5975', 'suberror': 'consent_required'}


In [66]:
import requests as rs
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
from azure.storage.blob import ContainerClient,BlobServiceClient
from io import StringIO
import numpy as np
import json
import datetime as dt
import warnings
import duckdb as db
import time

# Set display options

pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

cxn = db.connect('../DB/dbfile')

print( 'connected to Db')
time.sleep(1)

# Parse Secrets

try:
    secrets = json.load(open(file='../secrets.json'))
    print('Secrets file found.')
except:
    print('Secrets file not found or some error in file. Make sure the file exists in the same directory as the code.')

if secrets:
    refresh_token = secrets['refresh_token']

    app_id = secrets['app_id']

    app_secret = secrets['app_secret']

    request_body = {
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
            "scope": "openid",
            "resource": "https://management.core.windows.net/",
            "client_id": app_id,
            "client_secret": app_secret,
        }

    base_url = (
    'https://management.core.windows.net/'
    )


    isAccessTokenObtained = 0


try:
    req = rs.post(
                "https://login.windows.net/6e75cca6-47f0-47a3-a928-9d5315750bd9/oauth2/token"
                ,data = request_body)
    
    access_token = json.loads(req.text)['access_token']
    isAccessTokenObtained = 1
    print('Refresh Token valid, access token obtained. \nAccess token : ',access_token[:20]+'...')
except:
    print('Refresh token expired, or no internet connection. \nUnable to obtain access token')
    print(json.loads(req.text))
    

if isAccessTokenObtained == 1:
        
        HTTPheaders = {'Authorization': 'Bearer ' + access_token}

connected to Db
Secrets file found.
Refresh token expired, or no internet connection. 
Unable to obtain access token
{'error': 'invalid_grant', 'error_description': "AADSTS65001: The user or administrator has not consented to use the application with ID '006568e8-96fd-4d41-89cb-b90ad6d3ae7f' named 'Partner Center Web App'. Send an interactive authorization request for this user and resource. Trace ID: e3cece31-810f-43e8-84f3-a0a4ccfe1e00 Correlation ID: d6e4c91f-661f-4577-867c-2be54ea5cda6 Timestamp: 2024-02-15 14:11:49Z", 'error_codes': [65001], 'timestamp': '2024-02-15 14:11:49Z', 'trace_id': 'e3cece31-810f-43e8-84f3-a0a4ccfe1e00', 'correlation_id': 'd6e4c91f-661f-4577-867c-2be54ea5cda6', 'suberror': 'consent_required'}


In [8]:
iterate = False
cToken = None
iterationCount = 1
invoiceMonth = str(dt.datetime.now().strftime('%b')) + '_' + str((dt.datetime.now().year))

while iterate == True:
        
        print(f"iterationCount : {iterationCount}, cToken : {cToken}")

        HTTPheaders = {'Authorization': 'Bearer ' + access_token,
                        'MS-ContinuationToken': cToken if cToken else None}         # get cToken from response then pass it to the next call
        
        newCommerceOneTimeBillingURL = f'/v1/invoices/G038562576/lineitems?provider=onetime&invoiceLineItemType=usagelineitems&currencyCode=inr&period=current&size=5000' if\
        iterationCount == 1 else f'/v1/invoices/G038562576/lineitems?provider=onetime&invoiceLineItemType=usagelineitems&currencyCode=inr&period=current&size=5000&seekoperation=next'
        
        response = json.loads(
        rs.get(
                f"{base_url}{newCommerceOneTimeBillingURL}",
                headers=HTTPheaders)
        .content)
        
        df = pd.DataFrame(response['items'])

        cxn.query("truncate table azureconsumption") if iterationCount == 1 else None

        cxn.query(f"insert into azureConsumption select *,'{invoiceMonth}' as invoiceMonth from df")

        query("select count(1) from azureConsumption")
        
        try:
                cToken = response['continuationToken']            # get cToken from response
        except:
                break                                             # if no ctoken sent by api, break out of the loop

        iterationCount += 1





In [64]:
rid = 'd30b89b6-2fed-4f2b-ac4d-4221795b3f89'
url = 'https://management.azure.com/providers/Microsoft.Capacity/reservationorders/d30b89b6-2fed-4f2b-ac4d-4221795b3f89/providers/Microsoft.Consumption/reservationDetails?$filter={$filter}&api-version=2023-05-01'
                                                                                                                                                          
response = json.loads(
rs.get(
        url,
        headers=HTTPheaders)
.content)

response





{'error': {'code': 'InvalidAuthenticationTokenAudience',
  'message': "The access token has been obtained for wrong audience or resource 'https://api.partnercenter.microsoft.com'. It should exactly match with one of the allowed audiences 'https://management.core.windows.net/','https://management.core.windows.net','https://management.azure.com/','https://management.azure.com'."}}

In [43]:
query("select distinct customername from azureconsumption")

┌─────────────────────────────────────────────────┐
│                  customerName                   │
│                     varchar                     │
├─────────────────────────────────────────────────┤
│ Default Directory                               │
│ Intelogixs Research Private Limited             │
│ CRESCENT TAX COMPLIANCE PRIVATE LIMITED         │
│ Ncourage Social Enterprise Foundation           │
│ Hyderabad Institute of Neuro Sciences Pvt Ltd   │
│ NOMUS COMM-SYSTEMS PVT. LTD                     │
│ Infusion Solutions                              │
│ Ladida                                          │
│ Tata Chemicals Limited                          │
│ eSwiftSoftware                                  │
│ briodevptm                                      │
│ Elico Healthcare Services Ltd.                  │
│ Barbeque Nation Hospitality Ltd                 │
│ Biba Apparels Pvt Ltd                           │
│ Draup Business Solutions Private Limited        │
│ SOUTHERN S

# Daily rated from Ubed

In [82]:
query("""select customername, customerdomainname, entitlementid, entitlementdescription,
      round(sum(billingPreTaxTotal),2) amount,
      round(sum(billingPreTaxTotal)/0.85,2) MarginAmount15perc,
       round(amount+(0.18*amount)) TaxAmount18perc
       from ubed 
      where customername in ('Tata Chemicals Limited' ,'TATA CHEMICALS LIMITED')
      group by customername, customerdomainname, entitlementid, entitlementdescription
      order by 1,2,3
       """)

┌────────────────────────┬───────────────────────────────────┬──────────────────────────────────────┬───────────────────────────────────────────────────────┬───────────┬────────────────────┬─────────────────┐
│      CustomerName      │        CustomerDomainName         │            EntitlementId             │                EntitlementDescription                 │  amount   │ MarginAmount15perc │ TaxAmount18perc │
│        varchar         │              varchar              │               varchar                │                        varchar                        │  double   │       double       │     double      │
├────────────────────────┼───────────────────────────────────┼──────────────────────────────────────┼───────────────────────────────────────────────────────┼───────────┼────────────────────┼─────────────────┤
│ TATA CHEMICALS LIMITED │ tatachemicals.com                 │ 071707a3-865e-40dd-84f7-27337e9f8b94 │ Azure Subscription - Shiny Software For Truck Owners' │  15029

# Daily Rated from API

In [81]:
query("""select customername, customerdomainname, entitlementid, entitlementdescription,
      round(sum(billingPreTaxTotal),2) amount,
      round(sum(billingPreTaxTotal)/0.85,2) MarginAmount15perc,
       round(amount+(0.18*amount)) TaxAmount18perc
       from azureconsumption 
      where customername in ('Tata Chemicals Limited' ,'TATA CHEMICALS LIMITED')
      group by customername, customerdomainname, entitlementid, entitlementdescription
      order by 1,2,3
       """)

┌────────────────────────┬───────────────────────────────────┬──────────────────────────────────────┬───────────────────────────────────────────────────────┬───────────┬────────────────────┬─────────────────┐
│      customerName      │        customerDomainName         │            entitlementId             │                entitlementDescription                 │  amount   │ MarginAmount15perc │ TaxAmount18perc │
│        varchar         │              varchar              │               varchar                │                        varchar                        │  double   │       double       │     double      │
├────────────────────────┼───────────────────────────────────┼──────────────────────────────────────┼───────────────────────────────────────────────────────┼───────────┼────────────────────┼─────────────────┤
│ TATA CHEMICALS LIMITED │ tatachemicals.com                 │ 071707a3-865e-40dd-84f7-27337e9f8b94 │ Azure Subscription - Shiny Software For Truck Owners' │  15029

# Recon from API

In [54]:
#df = pd.DataFrame(getOneTimeInvoiceLineItems('G038562576'))

In [34]:
query("""select
            customerdomainname,
            round(sum(subtotal),2) amount,
            round(amount/0.85,2) marginAmount15perc,
            round(amount/0.82,2) totalAmount18perc
       from 
      df 
      where lower(customername) like '%tata%' group by customerdomainname""")

┌───────────────────────────────────┬───────────┬────────────────────┬───────────────────┐
│        customerDomainName         │  amount   │ marginAmount15perc │ totalAmount18perc │
│              varchar              │  double   │       double       │      double       │
├───────────────────────────────────┼───────────┼────────────────────┼───────────────────┤
│ tatachemicals.net                 │ 368282.99 │          433274.11 │          449125.6 │
│ tatachemicalslims.onmicrosoft.com │ 199361.83 │          234543.33 │         243124.18 │
│ tatachemicals.com                 │ 662516.95 │          779431.71 │          807947.5 │
└───────────────────────────────────┴───────────┴────────────────────┴───────────────────┘



In [55]:
query("select * from df where lower(customername) like '%reo%' and  invoicenumber = 'G038562576' and lower(skuname)  like '%reser%' ")

┌──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────┬─────────────────────────────────┬─────────────────┬───────────────┬─────────┬───────────────┬───────────────────────────────────┬──────────────────────────────┬──────────────┬─────────┬────────────────┬───────────────────────────────┬──────────────────────────────────────────────────────────────┬───────────────────┬─────────────┬───────────┬────────────────────┬──────────┬──────────┬──────────┬──────────┬──────────────────┬──────────┬───────────────┬─────────────┬──────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬─────────────────────┬──────────────┬──────────────────────────────────────┬────────────────────────────┬─────────────────┬─────────────────┬────────────────────┬────────────────────────┬───────────

In [47]:
query("select distinct customername from df order by 1")

┌────────────────────────────────────────────────────────────────┐
│                          customerName                          │
│                            varchar                             │
├────────────────────────────────────────────────────────────────┤
│     Jupiter  Lifeline Hospital Ltd                             │
│  Proov Identity Services Pvt. Ltd                              │
│ 1POINT1 SOLUTIONS                                              │
│ 1to1help.net                                                   │
│ 20Cube Logistics Solutions Private Limited                     │
│ 2b Sourcing                                                    │
│ 360BIM Solutions                                               │
│ 3S Engineering Solutions                                       │
│ 4seer Technologies Pvt Limited                                 │
│ 7Rays Semiconductors Pvt Ltd                                   │
│ 95 Techno                                                   

In [ ]:
query("select sum(totalForCustomer) from onetime where ")

In [22]:
# df = pd.DataFrame(getOneTimeInvoiceLineItems('G038562576'))

In [70]:
cxn.close()

In [1]:
import duckdb

In [38]:
duckdb.query("with names as (select customername  from read_parquet('../DB/onetime_invoices/*' )   group by customername having max(subtotal) = 0.0 and min(subtotal) = 0.0 and not (productid like 'CFQ%' or prodctid like 'DG%') ) select distinct customername,customerid from  read_parquet('../DB/onetime_invoices/*' ) where customername not in (select customername from names)").to_csv('../ubed_cx_with_billing.csv')

In [31]:
query("select * from read_parquet('../DB/onetime_invoices/*') where customername like 'Padiyar%'  ")

┌──────────────────────────────────────┬──────────────────────────────────────┬────────────────────┬─────────────────────────────────────┬─────────────────┬───────────────┬─────────┬───────────────┬───────────────────────────────────┬──────────────────────────────┬──────────────┬─────────┬────────────────┬─────────────────────┬───────────────────────────────────┬───────────────────┬────────────┬───────────┬────────────────────┬────────────┬──────────┬──────────┬──────────┬──────────────────┬──────────┬───────────────┬─────────────┬─────────────────────────┬──────────────────────────────────────┬──────────────────────┬──────────────────────┬────────────────────────────┬──────────────┬──────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────┬─────────────────┬─────────────────┬────────────────────┬────────────────────────┬──────────────────┬──────────────────────────────────────────────────────────────┬──────────────────┬──────